In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [3]:
import os
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
%run driving_data.ipynb

Instructions for updating:
non-resource variables are not supported in the long term


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive


In [4]:
!ls

MyDrive  Shareddrives


In [5]:
%cd /content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [6]:
%run model.ipynb

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
from tensorflow.core.protobuf import saver_pb2

In [8]:
LOGDIR = './save'

sess = tf.InteractiveSession()

L2NormConst = 0.001

train_vars = tf.trainable_variables()

loss = tf.reduce_mean(tf.square(tf.subtract(y_, y))) + tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
sess.run(tf.global_variables_initializer())

# create a summary to monitor cost tensor
tf.summary.scalar("loss", loss)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

saver = tf.train.Saver(write_version = saver_pb2.SaverDef.V2)

# op to write logs to Tensorboard
logs_path = './logs'
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

In [9]:
pip install pyswarms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.0 MB/s eta 0:00:00


In [10]:
hyperparam_dict = {
    'epochs': [],
    'batch_size': [],
    'train_loss': [],
    'val_loss': [],
    'test_loss': []
}

In [ ]:
import numpy as np
import pyswarms as ps

# Define objective function
def objective_func(params):
    # print(params)
    epochs = int(params[0][0])
    batch_size = int(params[1][0])
    hyperparam_dict['epochs'].append(epochs)
    hyperparam_dict['batch_size'].append(batch_size)
    print()
    print()
    print()
    print('epochs:')
    print(epochs)
    print('batch_size:')
    print(batch_size)
    
    # train over the dataset for specified number of epochs and batch size
    for epoch in range(epochs):
        for i in range(int(num_images/batch_size)):
            xs, ys = LoadTrainBatch(batch_size)
            feed_dict_2={x: xs, y_: ys, keep_prob: 0.8}
            train_step.run(feed_dict=feed_dict_2)

            if i % 10 == 0:
                xs_train, ys_train = LoadTrainBatch(batch_size)
                train_loss = loss.eval(feed_dict={x:xs_train, y_: ys_train, keep_prob: 1.0})
                xs_val, ys_val = LoadValBatch(batch_size)
                val_loss = loss.eval(feed_dict={x:xs_val, y_: ys_val, keep_prob: 1.0})
                print("Epoch: %d, Step: %d, Validation Loss: %g" % (epoch, epoch * batch_size + i, val_loss))

            # write logs at every iteration
            # summary = merged_summary_op.eval(feed_dict={x:xs, y_: ys, keep_prob: 1.0})
            # summary_writer.add_summary(summary, epoch * num_images/batch_size + i)

            if i % batch_size == 0:
                if not os.path.exists(LOGDIR):
                    os.makedirs(LOGDIR)
                checkpoint_path = os.path.join(LOGDIR, "model.ckpt")
                filename = saver.save(sess, checkpoint_path)

    hyperparam_dict['train_loss'].append(train_loss)
    hyperparam_dict['val_loss'].append(val_loss)
    # calculate and return testing loss
    test_loss = 0.0
    num_test_batches = int(num_test_images/batch_size)

    for i in range(num_test_batches):
        xs_test, ys_test = LoadTestBatch(batch_size)
        loss_value = loss.eval(feed_dict={x:xs_test, y_: ys_test, keep_prob: 1.0})
        test_loss += loss_value

    test_loss /= num_test_batches
    hyperparam_dict['test_loss'].append(test_loss)
    return test_loss

# Set the hyperparameter search space
bounds = (np.array([30, 30]), np.array([60, 100]))

# Set the PSO parameters
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

# Create the optimizer instance
optimizer = ps.single.GlobalBestPSO(n_particles=10, dimensions=2, options=options, bounds=bounds)

# Optimize the hyperparameters
best_cost, best_params = optimizer.optimize(objective_func, iters=5)

# Print the optimal hyperparameters
print('Optimal hyperparameters: epochs = {}, batch_size = {}'.format(int(best_params[0]), int(best_params[1])))

2023-04-30 05:56:46,447 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/5

Streaming output truncated to the last 5000 lines.
Epoch: 0, Step: 880, Validation Loss: 3.47977
Epoch: 0, Step: 890, Validation Loss: 3.34469
Epoch: 0, Step: 900, Validation Loss: 3.39706
Epoch: 0, Step: 910, Validation Loss: 3.42444
Epoch: 0, Step: 920, Validation Loss: 3.3402
Epoch: 0, Step: 930, Validation Loss: 3.39444
Epoch: 0, Step: 940, Validation Loss: 3.25122
Epoch: 0, Step: 950, Validation Loss: 3.21307
Epoch: 0, Step: 960, Validation Loss: 3.58586
Epoch: 0, Step: 970, Validation Loss: 3.5589
Epoch: 0, Step: 980, Validation Loss: 3.18412
Epoch: 0, Step: 990, Validation Loss: 3.38244
Epoch: 0, Step: 1000, Validation Loss: 3.2458
Epoch: 0, Step: 1010, Validation Loss: 3.10695
Epoch: 0, Step: 1020, Validation Loss: 3.12464
Epoch: 0, Step: 1030, Validation Loss: 3.36888
Epoch: 0, Step: 1040, Validation Loss: 3.36099
Epoch: 0, Step: 1050, Validation Loss: 3.26652
Epoch: 1, Step: 43, Validation Loss: 3.11822
Epoch: 1, Step: 53, Validation Loss: 4.95615
Epoch: 1, Step: 63, Validati

pyswarms.single.global_best:  20%|██        |1/5, best_cost=0.0888

Streaming output truncated to the last 5000 lines.
Epoch: 0, Step: 960, Validation Loss: 0.0513831
Epoch: 0, Step: 970, Validation Loss: 0.0506902
Epoch: 0, Step: 980, Validation Loss: 0.0505852
Epoch: 0, Step: 990, Validation Loss: 0.0495094
Epoch: 0, Step: 1000, Validation Loss: 0.0514733
Epoch: 0, Step: 1010, Validation Loss: 0.0522954
Epoch: 0, Step: 1020, Validation Loss: 0.0508855
Epoch: 0, Step: 1030, Validation Loss: 0.0500099
Epoch: 1, Step: 44, Validation Loss: 0.0571259
Epoch: 1, Step: 54, Validation Loss: 0.0534937
Epoch: 1, Step: 64, Validation Loss: 0.0531649
Epoch: 1, Step: 74, Validation Loss: 0.0495791
Epoch: 1, Step: 84, Validation Loss: 0.0639112
Epoch: 1, Step: 94, Validation Loss: 0.0504132
Epoch: 1, Step: 104, Validation Loss: 0.0514172
Epoch: 1, Step: 114, Validation Loss: 0.241055
Epoch: 1, Step: 124, Validation Loss: 0.0506005
Epoch: 1, Step: 134, Validation Loss: 0.0516583
Epoch: 1, Step: 144, Validation Loss: 0.0513285
Epoch: 1, Step: 154, Validation Loss: 0.

pyswarms.single.global_best:  40%|████      |2/5, best_cost=0.0462




epochs:
49
batch_size:
47
Epoch: 0, Step: 0, Validation Loss: 0.0298569
Epoch: 0, Step: 10, Validation Loss: 0.0498275
Epoch: 0, Step: 20, Validation Loss: 0.0374102
Epoch: 0, Step: 30, Validation Loss: 0.0361517
Epoch: 0, Step: 40, Validation Loss: 0.0345509
Epoch: 0, Step: 50, Validation Loss: 0.0294084
Epoch: 0, Step: 60, Validation Loss: 0.0287215
Epoch: 0, Step: 70, Validation Loss: 0.0292929
Epoch: 0, Step: 80, Validation Loss: 0.032281
Epoch: 0, Step: 90, Validation Loss: 0.0733647
Epoch: 0, Step: 100, Validation Loss: 0.0351573
Epoch: 0, Step: 110, Validation Loss: 0.0403288
Epoch: 0, Step: 120, Validation Loss: 0.0423005
Epoch: 0, Step: 130, Validation Loss: 0.0387385
Epoch: 0, Step: 140, Validation Loss: 0.039244
Epoch: 0, Step: 150, Validation Loss: 0.0849518
Epoch: 0, Step: 160, Validation Loss: 0.030959
Epoch: 0, Step: 170, Validation Loss: 0.039
Epoch: 0, Step: 180, Validation Loss: 0.0290979
Epoch: 0, Step: 190, Validation Loss: 0.032869
Epoch: 0, Step: 200, Validatio

In [1]:
import pandas as pd
hyperparam_df = pd.DataFrame.from_dict(hyperparam_dict)

NameError: ignored

In [2]:
hyperparam_df

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import pandas as pd

# Create a three-dimensional plot for Test Loss
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot(hyperparam_df['epochs'], hyperparam_df['batch_size'], hyperparam_df['test_loss'])
ax.set_xlabel('Epochs')
ax.set_ylabel('Batch Size')
ax.set_zlabel('Test Loss')
plt.show()

In [3]:
# Create a three-dimensional plot for Validation Loss
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot(hyperparam_df['epochs'], hyperparam_df['batch_size'], hyperparam_df['val_loss'])
ax.set_xlabel('Epochs')
ax.set_ylabel('Batch Size')
ax.set_zlabel('Val Loss')
plt.show()

NameError: ignored

In [4]:
# Create a three-dimensional plot for Training Loss
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot(hyperparam_df['epochs'], hyperparam_df['batch_size'], hyperparam_df['train_loss'])
ax.set_xlabel('Epochs')
ax.set_ylabel('Batch Size')
ax.set_zlabel('Train Loss')
plt.show()

NameError: ignored